In [8]:
using Distributions, TimeIt, ProgressMeter, PyPlot, JLD
include("/home/postdoc/dsen/Desktop/codes/G-ZZ_clean/jl_files/zz_samplers.jl")
include("temp.jl")

run_sampler (generic function with 4 methods)

In [9]:
d, Nobs = 251, 1000
pX = 100/d
pξ = 1e-1
X = sprandn(d, Nobs, pX)
X[1,:] = 1
ξ_true = sprandn(d, pξ)
y = Float64.([rand(Binomial(1, 1/(1+exp(-ξ_true'X[:,j])))) for j in 1:Nobs])
save("shrinkage_data_2.01.new.jld", "X", X, "y", y)

In [10]:
X, y = load("shrinkage_data_2.01.new.jld", "X", "y")
d, Nobs = size(X)
σ02 = 1
my_pr = SS_prior(d, σ02)
my_ll = ll_logistic_sp(X,y)
my_model = model(my_ll, my_pr);

### GZZ:

In [11]:
# Sub-sampling without control variates and with weights:
mb_size = 1
max_attempts = 5*10^5
lambda = 2e1
Print = true;

In [12]:
start = time()
run_sampler(my_model, lambda, max_attempts, mb_size, Print) 
print("Sampling in ", round((time()-start)/60,1), " mins \n")

10 percent attempts in 0.62 min; zz bounces = 11012, hyp bounces = 5, total time of process = 0.601 
20 percent attempts in 1.31 min; zz bounces = 20724, hyp bounces = 20, total time of process = 1.201 
30 percent attempts in 2.01 min; zz bounces = 29526, hyp bounces = 32, total time of process = 1.800 
40 percent attempts in 2.71 min; zz bounces = 37484, hyp bounces = 41, total time of process = 2.404 
50 percent attempts in 3.40 min; zz bounces = 44830, hyp bounces = 54, total time of process = 3.005 
60 percent attempts in 4.08 min; zz bounces = 51627, hyp bounces = 71, total time of process = 3.602 
70 percent attempts in 4.76 min; zz bounces = 58250, hyp bounces = 83, total time of process = 4.200 
80 percent attempts in 5.43 min; zz bounces = 64601, hyp bounces = 93, total time of process = 4.801 
90 percent attempts in 6.11 min; zz bounces = 70925, hyp bounces = 105, total time of process = 5.405 
100 percent attempts in 6.78 min; zz bounces = 77060, hyp bounces = 123, total tim

In [13]:
fig = figure("pyplot_barplot",figsize=(5,3))
n_samples = 10^4
maxlag_gzz = 500
acfs_gzz = zeros(d,maxlag_gzz)
discard = 0
filename  = "/xtmp/GZZ_data/shrinkage_prior/synthetic_data/lambda:"*string(lambda)*
            "-d:"*string(d)*"-Nobs:"*string(Nobs)*"-mb_size:"*string(mb_size)*".jld"
xt_skel, bt_skel = load(filename, "xt_skeleton", "bt_skeleton")
xi_samples_gzz = extract_samples(xt_skel[:,discard+1:end], 
                             bt_skel[:,discard+1:end], 
                            (bt_skel[end]-bt_skel[discard+1])/n_samples,
                            "linear")[:,1:end-1]
iacts_gzz = zeros(size(xi_samples_gzz,1))
for dim in 1:size(xi_samples_gzz,1)
    a = acf(xi_samples_gzz[dim,:],maxlag_gzz)
    plot(a)
    acfs_gzz[dim,:] = a
end
title("ACFs for GZZ")
xlabel("Lag")
ylabel("ACF")
grid(true)
xt_skel, bt_skel, xi_samples = [], [], []
gc()

LoadError: [91mDimensionMismatch("tried to assign 501-element array to 1×500 destination")[39m